<a href="https://colab.research.google.com/github/caballerofelipe/Llama-Tech-Talks/blob/main/Ejemplo_Qlora_gpu_A100_with_outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ejecutado en Colab con GPU A100.

# Instalar lo que falta

In [ ]:
!date

Fri Aug  8 12:47:18 AM UTC 2025


In [ ]:
!pip install bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!date

Fri Aug  8 12:48:52 AM UTC 2025


# Ingresar a Hugging Face

In [ ]:
# Agregar HF_TOKEN a los secretos de Colab o como variable de entorno
from google.colab import userdata
import os
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

# Importar librerías

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training
import torch
import time
import json
from datasets import Dataset, load_dataset
from trl import SFTTrainer

# Preparación del modelo

## Configurar la cuantización

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    )


## Configuración de LoRA

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)


## Crear el modelo con Cuantización

In [ ]:
# device = "auto"
device = "cuda:0"
# device = "cpu"

# model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_id = "meta-llama/Llama-3.2-3B-Instruct"
model_id = "meta-llama/Llama-3.2-1B-Instruct"

model_name = model_id.split('/')[1]

In [ ]:
start_time = time.time()

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    quantization_config=bnb_config,
    # torch_dtype=torch.bfloat16,
    )

# https://github.com/huggingface/trl/issues/3683#issuecomment-3046420837
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)

# Needed with Cuda, https://stackoverflow.com/a/79494162/1071459
model.enable_input_require_grads()

# Aplicar LoRA
model = get_peft_model(model, peft_config)

# Muestra parámetros entrenables
model.print_trainable_parameters()

load_time = time.time() - start_time
print(f"✅ Modelo cargado en {load_time:.2f}s")

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

trainable params: 11,272,192 || all params: 1,247,086,592 || trainable%: 0.9039
✅ Modelo cargado en 17.06s


In [ ]:
!date

Fri Aug  8 12:49:29 AM UTC 2025


# Preparar el Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

tokenizer.vocab_size

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

128000

In [ ]:
!date

Fri Aug  8 12:49:31 AM UTC 2025


# Preparar data

In [ ]:
# Dataset de ejemplo para chatbot de soporte técnico
sample_conversations = [
    {
        "messages": [
            {
                "role": "system",
                "content": "Eres un asistente de soporte técnico especializado en ayudar con problemas de software y hardware.",
            },
            {
                "role": "user",
                "content": "Mi computadora se reinicia sola, ¿qué puede estar pasando?",
            },
            {
                "role": "assistant",
                "content": "Los reinicios automáticos pueden tener varias causas. Las más comunes son: 1) Sobrecalentamiento del procesador o GPU, 2) Problemas con la fuente de poder, 3) Memoria RAM defectuosa, 4) Actualizaciones automáticas de Windows. Te recomiendo verificar las temperaturas con un software como HWMonitor y revisar el Visor de eventos de Windows para identificar errores específicos.",
            },
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "Eres un asistente de soporte técnico especializado en ayudar con problemas de software y hardware.",
            },
            {"role": "user", "content": "No puedo conectarme a WiFi desde mi laptop"},
            {
                "role": "assistant",
                "content": "Para resolver problemas de WiFi, sigue estos pasos: 1) Verifica que el WiFi esté activado en tu laptop (tecla Fn + tecla WiFi), 2) Reinicia el router desconectándolo 30 segundos, 3) Olvida y vuelve a conectar la red WiFi, 4) Actualiza los drivers de la tarjeta de red desde el Administrador de dispositivos, 5) Ejecuta el solucionador de problemas de red de Windows. Si persiste el problema, puede ser un issue de hardware.",
            },
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "Eres un asistente de soporte técnico especializado en ayudar con problemas de software y hardware.",
            },
            {"role": "user", "content": "Mi aplicación se cierra inesperadamente"},
            {
                "role": "assistant",
                "content": "Los cierres inesperados de aplicaciones pueden deberse a: 1) Falta de memoria RAM disponible, 2) Conflictos con otros programas, 3) Archivos de la aplicación corruptos, 4) Drivers desactualizados. Soluciones: Cierra otros programas para liberar memoria, ejecuta la aplicación como administrador, reinstala la aplicación, actualiza drivers gráficos y verifica si hay actualizaciones de la aplicación.",
            },
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "Eres un asistente de soporte técnico especializado en ayudar con problemas de software y hardware.",
            },
            {"role": "user", "content": "¿Cómo puedo acelerar mi computadora lenta?"},
            {
                "role": "assistant",
                "content": "Para acelerar una computadora lenta: 1) Ejecuta Liberador de espacio en disco para eliminar archivos temporales, 2) Desinstala programas que no uses, 3) Desactiva programas de inicio innecesarios en el Administrador de tareas, 4) Ejecuta un antivirus completo, 5) Considera agregar más RAM si tienes menos de 8GB, 6) Reemplaza el disco duro por un SSD si aún usas HDD, 7) Actualiza drivers y el sistema operativo.",
            },
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "Eres un asistente de soporte técnico especializado en ayudar con problemas de software y hardware.",
            },
            {"role": "user", "content": "No puedo imprimir documentos"},
            {
                "role": "assistant",
                "content": "Para resolver problemas de impresión: 1) Verifica que la impresora esté encendida y conectada, 2) Revisa que haya papel y tinta/tóner suficiente, 3) Reinicia el spooler de impresión (services.msc > Spooler de impresión > Reiniciar), 4) Actualiza o reinstala los drivers de la impresora, 5) Configura la impresora como predeterminada, 6) Ejecuta el solucionador de problemas de impresión de Windows. Si es una impresora de red, verifica la conectividad.",
            },
        ]
    },
]

# Expandir dataset con variaciones
expanded_conversations = []

# Agregar conversaciones originales
expanded_conversations.extend(sample_conversations)

# Agregar variaciones con diferentes tonos
for conv in sample_conversations:
    # Versión más técnica
    tech_conv = conv.copy()
    tech_conv["messages"] = conv["messages"].copy()
    tech_conv["messages"][0] = {
        "role": "system",
        "content": "Eres un especialista técnico senior con amplia experiencia en diagnóstico y resolución de problemas de TI.",
    }
    expanded_conversations.append(tech_conv)

    # Versión más amigable
    friendly_conv = conv.copy()
    friendly_conv["messages"] = conv["messages"].copy()
    friendly_conv["messages"][0] = {
        "role": "system",
        "content": "Eres un asistente de soporte técnico amigable que explica las soluciones de manera simple y comprensible.",
    }
    expanded_conversations.append(friendly_conv)

# Guardar dataset
save_path= "sample_chatbot_data.json"
with open(save_path, 'w', encoding='utf-8') as f:
    json.dump(expanded_conversations, f, indent=2, ensure_ascii=False)

In [ ]:
data_path = "sample_chatbot_data.json"
with open(data_path, 'r', encoding='utf-8') as f:
    conversations = json.load(f)
len(conversations), conversations[-1]

(15,
 {'messages': [{'role': 'system',
    'content': 'Eres un asistente de soporte técnico amigable que explica las soluciones de manera simple y comprensible.'},
   {'role': 'user', 'content': 'No puedo imprimir documentos'},
   {'role': 'assistant',
    'content': 'Para resolver problemas de impresión: 1) Verifica que la impresora esté encendida y conectada, 2) Revisa que haya papel y tinta/tóner suficiente, 3) Reinicia el spooler de impresión (services.msc > Spooler de impresión > Reiniciar), 4) Actualiza o reinstala los drivers de la impresora, 5) Configura la impresora como predeterminada, 6) Ejecuta el solucionador de problemas de impresión de Windows. Si es una impresora de red, verifica la conectividad.'}]})

In [ ]:
formatted_data = []

for conv in conversations:
    # Aplicar chat template
    formatted_text = tokenizer.apply_chat_template(
        conv["messages"], tokenize=False, add_generation_prompt=False
    )

    formatted_data.append({"text": formatted_text})

formatted_data[-1]

{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 08 Aug 2025\n\nEres un asistente de soporte técnico amigable que explica las soluciones de manera simple y comprensible.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nNo puedo imprimir documentos<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nPara resolver problemas de impresión: 1) Verifica que la impresora esté encendida y conectada, 2) Revisa que haya papel y tinta/tóner suficiente, 3) Reinicia el spooler de impresión (services.msc > Spooler de impresión > Reiniciar), 4) Actualiza o reinstala los drivers de la impresora, 5) Configura la impresora como predeterminada, 6) Ejecuta el solucionador de problemas de impresión de Windows. Si es una impresora de red, verifica la conectividad.<|eot_id|>'}

In [ ]:
# Crear dataset
dataset = Dataset.from_list(formatted_data)

# Dividir en train/validation
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"📊 Dataset dividido:")
print(f"   Entrenamiento: {len(train_dataset)} ejemplos")
print(f"   Validación: {len(eval_dataset)} ejemplos")

# Mostrar ejemplo
print(f"\n📝 Ejemplo de entrada formateada:")
# print(f"{train_dataset[0]['text'][:200]}...")
print(f"{train_dataset[0]['text']}...")

📊 Dataset dividido:
   Entrenamiento: 12 ejemplos
   Validación: 3 ejemplos

📝 Ejemplo de entrada formateada:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 08 Aug 2025

Eres un asistente de soporte técnico especializado en ayudar con problemas de software y hardware.<|eot_id|><|start_header_id|>user<|end_header_id|>

¿Cómo puedo acelerar mi computadora lenta?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Para acelerar una computadora lenta: 1) Ejecuta Liberador de espacio en disco para eliminar archivos temporales, 2) Desinstala programas que no uses, 3) Desactiva programas de inicio innecesarios en el Administrador de tareas, 4) Ejecuta un antivirus completo, 5) Considera agregar más RAM si tienes menos de 8GB, 6) Reemplaza el disco duro por un SSD si aún usas HDD, 7) Actualiza drivers y el sistema operativo.<|eot_id|>...


# Entrenamiento

In [ ]:
output_dir=f'{model_name}_finetuned'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=10,
    eval_steps=50,
    save_steps=100,
    eval_strategy="steps",
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to='none',  # Desactivar wandb por defecto
    # bf16=True,
    use_cpu=(False if device != 'cpu' else True),
    # gradient_checkpointing=True,
    dataloader_pin_memory=False,
    remove_unused_columns=False,
)

In [ ]:
!date

Fri Aug  8 12:49:31 AM UTC 2025


In [ ]:
trainer = SFTTrainer(
    # model=model.to(device),
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=training_args,
    #
    # max_seq_length=2048,
    # packing=False,
    # dataset_text_field="text",
)

print("✅ Trainer configurado")
print('Dispositivo de entrenamiento:', trainer.args.device)

# Iniciar entrenamiento
print("🎯 Iniciando entrenamiento...")
start_time = time.time()

train_result = trainer.train()

training_time = time.time() - start_time

print(f"✅ Entrenamiento completado en {training_time/60:.2f} minutos")

# Guardar modelo
print("💾 Guardando modelo...")
trainer.save_model(f'{model_name}_finetuned.save_model')
tokenizer.save_pretrained(f'{model_name}_finetuned.save_pretrained')

# Guardar métricas
metrics = {
            "training_time_minutes": training_time / 60,
            "final_train_loss": train_result.training_loss,
            "training_steps": train_result.global_step,
            "model_name": model_id,
            "peft_config": {
                "r": peft_config.r,
                "lora_alpha": peft_config.lora_alpha,
                "lora_dropout": peft_config.lora_dropout,
            },
            "training_args": {
                "num_epochs": training_args.num_train_epochs,
                "batch_size": training_args.per_device_train_batch_size,
                "learning_rate": training_args.learning_rate,
            },
        }
metrics

with open(f"{output_dir}/training_metrics.json", 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"🎉 Modelo fine-tuned guardado en: {output_dir}")

Adding EOS to train dataset:   0%|          | 0/12 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/12 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/12 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

✅ Trainer configurado
Dispositivo de entrenamiento: cuda:0
🎯 Iniciando entrenamiento...


Step,Training Loss,Validation Loss


✅ Entrenamiento completado en 0.09 minutos
💾 Guardando modelo...
🎉 Modelo fine-tuned guardado en: Llama-3.2-1B-Instruct_finetuned


In [ ]:
!date

Fri Aug  8 12:49:39 AM UTC 2025


# Resulados de prueba

In [ ]:
def model_generate_response(messages):
    # Aplicar chat template
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Tokenizar
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(device)
    # print(inputs)

    # Generar
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.1,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )

    # Decodificar
    response = tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1] :], skip_special_tokens=True
    )

    return response.strip()

In [ ]:
# if test_prompts is None:
test_prompts = [
    "Mi computadora no enciende, ¿qué puedo hacer?",
    "¿Cómo puedo recuperar archivos borrados accidentalmente?",
    "Mi internet va muy lento, ¿cómo lo soluciono?",
    "La pantalla de mi laptop está parpadeando",
    "No puedo instalar un programa, me da error",
]

print(f"🧪 Evaluando con {len(test_prompts)} prompts de prueba...")

results = []

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n🔍 Test {i}/{len(test_prompts)}:")
    print(f"Prompt: {prompt}")

    # Crear conversación de prueba
    messages = [
        {
            "role": "system",
            "content": "Eres un asistente de soporte técnico especializado en ayudar con problemas de software y hardware.",
        },
        {"role": "user", "content": prompt},
    ]

    # Generar respuesta
    response = model_generate_response(messages)

    print(f"Respuesta: {response}")

    results.append({"prompt": prompt, "response": response, "length": len(response.split())})

    print("-" * 50)

# Análisis de resultados
avg_length = sum(r["length"] for r in results) / len(results)
print(f"\n📊 Estadísticas de evaluación:")
print(f"   Longitud promedio respuesta: {avg_length:.1f} palabras")
print(f"   Respuestas generadas: {len(results)}")

# Guardar resultados
with open(f"{output_dir}/evaluation_results.json", 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"✅ Resultados guardados en: {output_dir}/evaluation_results.json")

🧪 Evaluando con 5 prompts de prueba...

🔍 Test 1/5:
Prompt: Mi computadora no enciende, ¿qué puedo hacer?
Respuesta: Lo siento, pero parece que hubo un pequeño error en tu pregunta. Como asistente de soporte técnico, puedo ayudarte a diagnosticar y solucionar problemas de software y hardware. Sin embargo, no puedo ayudarte a encender tu computadora.

Aquí te presento algunos pasos generales que puedes seguir para intentar encender tu computadora:

1. **Revisa la conexión**: Asegúrate de que la conexión a la red sea viva y que la puerta de la computadora esté abierta. Verifica que la placa base esté conectada a la computadora y que no haya cables extraños en el lugar.
2. **Verifica la alimentación
--------------------------------------------------

🔍 Test 2/5:
Prompt: ¿Cómo puedo recuperar archivos borrados accidentalmente?
Respuesta: Para recuperar archivos borrados accidentalmente, sigue estos pasos:

**Reconocimiento de la ubicación y contenido del archivo borrado**

1.  Identifica l

In [ ]:
print("📥 Cargando modelo base...")
base_tokenizer = AutoTokenizer.from_pretrained(model_id)
if base_tokenizer.pad_token is None:
    base_tokenizer.pad_token = base_tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map=device,
)

print("✅ Modelo base cargado")

📥 Cargando modelo base...
✅ Modelo base cargado


In [ ]:
def base_model_generate_response(messages):
    # Aplicar chat template
    prompt = base_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Tokenizar
    inputs = base_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(device)
    # print(inputs)

    # Generar
    with torch.no_grad():
        outputs = base_model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.1,
            top_p=0.9,
            do_sample=True,
            pad_token_id=base_tokenizer.eos_token_id,
        )

    # Decodificar
    response = base_tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1] :], skip_special_tokens=True
    )

    return response.strip()

In [ ]:
# Comparar con modelo base
test_prompts = [
    "Mi computadora no enciende",
    "No puedo conectarme a internet",
    "La aplicación se cierra sola",
]

comparison_results = []

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n🔍 Comparación {i}/{len(test_prompts)}:")
    print(f"Prompt: {prompt}")

    messages = [
        {
            "role": "system",
            "content": "Eres un asistente de soporte técnico especializado en ayudar con problemas de software y hardware.",
        },
        {"role": "user", "content": prompt},
    ]

    base_response = base_model_generate_response(messages)
    finetuned_response = model_generate_response(messages)

    print(f"🤖 Modelo Base: {base_response}")
    print(f"🎯 Fine-tuned: {finetuned_response}")

    comparison_results.append(
        {
            "prompt": prompt,
            "base_response": base_response,
            "finetuned_response": finetuned_response,
            "base_length": len(base_response.split()),
            "finetuned_length": len(finetuned_response.split()),
        }
    )


🔍 Comparación 1/3:
Prompt: Mi computadora no enciende
🤖 Modelo Base: ¡Lo siento mucho! ¿Podría ser un problema de hardware o de software? Aquí te dejo algunos pasos que puedes seguir para intentar diagnosticar el problema:

**Pasos para diagnosticar el problema de hardware:**

1. **Verifica la alimentación**: Asegúrate de que la computadora esté conectada a la tomada de corriente y que el cable de alimentación esté conectado correctamente.
2. **Verifica el cable de datos**: Asegúrate de que el cable de datos esté conectado correctamente a la placa base y a la placa de tarjeta gráfica.
3. **Verifica el sistema operativo**: Asegúrate de
🎯 Fine-tuned: Lo siento, pero parece que mi capacidad para ayudarte se ha limitado. Por favor, proporciona más detalles sobre el problema que estás experimentando con tu computadora. ¿Podrías mencionar:

* ¿Cuándo intentaste iniciar la computadora?
* ¿Qué pasó exactamente cuando intentaste iniciar?
* ¿Hay algún error específico que aparezca en la pantall

In [ ]:
!date

Fri Aug  8 12:50:50 AM UTC 2025


Sitios Útiles:
- https://github.com/huggingface/peft